### Earthquake intensity zone statistics


In [1]:
### Method ###

##Output:
# Provinces with population per earthquake risk column. 
# 5 earthquake risk columns
# Shp or spatial database table

##Input:
# Province polygons
# 5 earthquake risk columns
# Shp or spatial database table

##Input:
# Province polygons
# Earthquake polyons
# Population raster

## Steps:

# 1.1
# Clip province polygons by earthquake polygons
# Explode polygons (multipart to single part)

# 2.1
# Extract population for each polygon using zonal statistics

# 2.2
# Extract building count for each polygon using spatial join and count

# 2.3
# Extract area for each polygon

# 3.1
# Summarize by both province, earthquake, building, area and intensity using pandas pivot table

# 4.1
# Join pivot table columns to original province polygons x3 (seperate for each dataset)

# 5.1
# QA


In [2]:
import os
import json
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 # required for exporting to postgis
import rioxarray as rxr
from rasterio.crs import CRS
from sqlalchemy import create_engine
import rasterstats
from shapely.ops import transform
from datetime import datetime

import matplotlib.pyplot as plt

### Setting connection and parameters

In [3]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# Define projection
repro_crs = '+proj=cea'

### Data

In [6]:
pcode = 'reg_pcode' ### OBS: Adjust based on admin level

pop = r'D:\iMMAP\data\Afghanistan\afg_worldpop_2020_UNadj_unconstrained_projCEA.tif'
adm = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_region_stats', con)
#adm = gpd.read_file(r"D:\iMMAP\data\Afghanistan\HSDC-Official\afg_admbnda_adm1.shp")#.to_crs(repro_crs)
eq = gpd.read_file(r"D:\iMMAP\data\Afghanistan\HSDC-Official\afg_eq_hzda.shp")#.to_crs(repro_crs)

print('    Loading buildings   Start: {}'.format(datetime.now().strftime("%H:%M:%S")))
build = gpd.GeoDataFrame.from_postgis('SELECT * from afg_buildings_microsoft_centroids', con).to_crs(repro_crs)
print('    Loading buildings   End  : {}'.format(datetime.now().strftime("%H:%M:%S")))

    Loading buildings   Start: 13:00:23
    Loading buildings   End  : 13:03:59


In [7]:
adm.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
eq.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### 1.1 Clip province polygons by earthquake polygons


In [9]:
# Use the overlay function to clip admin polygons by earthquake zones
overlay = gpd.overlay(adm, eq, how='intersection', keep_geom_type=None, make_valid=True)

# Post-process
adm_eq = overlay.explode()
adm_eq = adm_eq.to_crs(repro_crs)
adm_eq = adm_eq.reset_index()
adm_eq = adm_eq.drop(columns=['level_1', 'level_0'])

C:\Users\VMO\AppData\Local\Temp\ipykernel_24644\3196807887.py:5: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  adm_eq = overlay.explode()


### 2.1 Extract population for each polygon using zonal statistics

In [10]:
# Produce list of dictionaries, each dictionary representing the sum for one polygon
zonalSt = rasterstats.zonal_stats(adm_eq, pop, stats = 'sum')

In [11]:
# Convert list of dictionaries to pandas dataframe (containing one column with each value)
df = pd.DataFrame(zonalSt)
df = df.rename(columns={'sum': 'pop_eq'})

In [12]:
# Add the values to the original table
df_concat = pd.concat([df, adm_eq], axis=1)

In [13]:
# Convert table back into a geodataframe    
gdf = gpd.GeoDataFrame(df_concat, geometry=df_concat.geometry) #wkb_geometry

In [14]:
# Reorder columns so population sum is last
gdf_reordered_columns = gdf[[c for c in gdf if c not in ['pop_eq']] + ['pop_eq']]
stats = gdf_reordered_columns

### 2.2 Extract building count for each polygon using spatial join and count

In [15]:
#def drop_building_column(df):
#    if 'build' in df.columns:
#        df.drop('build', axis=1, inplace=True)
#    return df

In [16]:
# Join building centroids to admin polygons
print('    Joining points to polygons   Start: {}'.format(datetime.now().strftime("%H:%M:%S")))

points = build
polygons = stats

#drop_building_column(polygons)

# Joining the polygon attributes to each point
joined_df = gpd.sjoin(
    points,
    polygons,
    how='inner',
    predicate='intersects')

print('    Joining points to polygons   Finish: {}'.format(datetime.now().strftime("%H:%M:%S")))

    Joining points to polygons   Start: 13:04:10
    Joining points to polygons   Finish: 13:13:45


In [17]:
# Count number of buildings within admin polygons

print('    Counting number of buildings          Start: {}'.format(datetime.now().strftime("%H:%M:%S")))

build_count = joined_df.groupby(
    [pcode],
    as_index=False,
)['geom'].count() # column is arbitrary

# Change column name to build_count
build_count.rename(columns = {'geom': 'bld_eq'}, inplace = True)

print('    Counting number of buildings          End:   {}'.format(datetime.now().strftime("%H:%M:%S")))

    Counting number of buildings          Start: 13:13:45
    Counting number of buildings          End:   13:13:46


In [18]:
# Merge build count back on to admin dataset
polygons = polygons.merge(
    build_count, 
    on=pcode, 
    how='left')

In [19]:
polygons.sort_values('bld_eq')

,tid,id,reg_en,reg_da,reg_pcode,reg_ref,regalt1en,regalt2en,regalt1da,regalt2da,adm0_en,adm0_da,adm0_pcode,date,validon,validto,shape_leng_1,shape_area_1,pop,km2,sett,build,pop_density,pop_f029,km2_f029,build_f029,pop_f121,km2_f121,build_f121,pop_f271,km2_f271,build_f271,km2_av,pop_av,build_av,pop_ls_0,pop_ls_1,pop_ls_2,pop_ls_3,pop_ls_4,km2_ls_0,km2_ls_1,km2_ls_2,km2_ls_3,km2_ls_4,build_ls_0,build_ls_1,build_ls_2,build_ls_3,build_ls_4,ogc_fid,acc_val,valley,seismic_in,source,data,population,shape_leng_2,shape_area_2,seismic__1,intensity,geometry,pop_eq,bld_eq
25,2,2,Central Highland,لوړه مرکزی حوزه,CH,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.035766,3.100092,1224738,31641.50,4001,141838,38.71,48397,857.81,12470,35019,602.58,7939,16340,246.78,3357,3382.76,116303,17630,249666,58983,539781,271531,104775,3591.54,1473.89,16428.51,7309.05,2838.51,66744,8211,53643,12107,1133,19,34.0,0,"VIII. Severe shaking, furniture overturned, an...","By Oliver S. Boyd, Charles S. Mueller, and Ken...",Peak Horizontal Acceleration with 2 Percent Pr...,None,57.991642,12.775139,VIII,8,"POLYGON ((7594511.111 3636214.838, 7594643.551...",7.257501e+04,141838
50,2,2,Central Highland,لوړه مرکزی حوزه,CH,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.035766,3.100092,1224738,31641.50,4001,141838,38.71,48397,857.81,12470,35019,602.58,7939,16340,246.78,3357,3382.76,116303,17630,249666,58983,539781,271531,104775,3591.54,1473.89,16428.51,7309.05,2838.51,66744,8211,53643,12107,1133,3,9.2,0,VI. Strong shaking with few people losing thei...,"By Oliver S. Boyd, Charles S. Mueller, and Ken...",Peak Horizontal Acceleration with 2 Percent Pr...,None,34.304394,10.814954,VI,6,"POLYGON ((7354276.169 3464176.083, 7354381.485...",7.724805e+01,141838
49,2,2,Central Highland,لوړه مرکزی حوزه,CH,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.035766,3.100092,1224738,31641.50,4001,141838,38.71,48397,857.81,12470,35019,602.58,7939,16340,246.78,3357,3382.76,116303,17630,249666,58983,539781,271531,104775,3591.54,1473.89,16428.51,7309.05,2838.51,66744,8211,53643,12107,1133,3,9.2,0,VI. Strong shaking with few people losing thei...,"By Oliver S. Boyd, Charles S. Mueller, and Ken...",Peak Horizontal Acceleration with 2 Percent Pr...,None,34.304394,10.814954,VI,6,"POLYGON ((7369848.482 3489065.831, 7369921.428...",2.928568e+03,141838
48,2,2,Central Highland,لوړه مرکزی حوزه,CH,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.035766,3.100092,1224738,31641.50,4001,141838,38.71,48397,857.81,12470,35019,602.58,7939,16340,246.78,3357,3382.76,116303,17630,249666,58983,539781,271531,104775,3591.54,1473.89,16428.51,7309.05,2838.51,66744,8211,53643,12107,1133,3,9.2,0,VI. Strong shaking with few people losing thei...,"By Oliver S. Boyd, Charles S. Mueller, and Ken...",Peak Horizontal Acceleration with 2 Percent Pr...,None,34.304394,10.814954,VI,6,"POLYGON ((7289438.943 3542438.461, 7289472.053...",5.578412e+05,141838
4,2,2,Central Highland,لوړه مرکزی حوزه,CH,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.035766,3.100092,1224738,31641.50,4001,141838,38.71,48397,857.81,12470,35019,602.58,7939,16340,246.78,3357,3382.76,116303,17630,249666,58983,539781,271531,104775,3591.54,1473.89,16428.51,7309.05,2838.51,66744,8211,53643,12107,1133,17,9.2,0,VI. Strong shaking with few people losing thei...,"By Oliver S. Boyd, Charles S. Mueller, and Ken...",Peak Horizontal Acceleration with 2 Percent Pr...,None,19.902211,5.507107,VI,6,"POLYGON ((7412021.779 3620301.743, 7411954.234...",3.868621e+04,141838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,4,4,Southern,جنوبی حوزه,SR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,20

### 2.3 Extract area for each polygon

In [20]:
polygons['km2_eq'] = polygons['geometry'].area.div(1000000)

### 3.1 Summarize by both province and earthquake intensity using pandas pivot table

In [21]:
# pivot table to sum population by admin and intensity
pivoted = pd.pivot_table(polygons, values=['pop_eq','bld_eq', 'km2_eq'], index=pcode, columns='intensity', aggfunc='sum')

# fill NaN values with 0
pivoted.fillna(0, inplace=True)

# reset index to make admin a column again
pivoted = pivoted.reset_index()

### 4.1 Join pivot table columns to original province polygons

In [45]:
eq_stats = adm.merge(
            pivoted, 
            on=pcode, 
            how='left')

C:\Users\VMO\anaconda3\envs\earth2\lib\site-packages\geopandas\geodataframe.py:1470: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  result = DataFrame.merge(self, *args, **kwargs)
C:\Users\VMO\anaconda3\envs\earth2\lib\site-packages\geopandas\geodataframe.py:1470: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  result = DataFrame.merge(self, *args, **kwargs)


In [46]:
eq_stats

,id,shape_leng,shape_area,adm1_en,adm1_da,adm1_pcode,adm1_ref,adm1alt1en,adm1alt2en,adm1alt1da,adm1alt2da,adm0_en,adm0_da,adm0_pcode,date,validon,validto,reg_en,reg_da,reg_pcode,pop,km2,sett,build,pop_density,pop_f029,km2_f029,build_f029,pop_f121,km2_f121,build_f121,pop_f271,km2_f271,build_f271,km2_av,pop_av,build_av,pop_ls_0,pop_ls_1,pop_ls_2,pop_ls_3,pop_ls_4,km2_ls_0,km2_ls_1,km2_ls_2,km2_ls_3,km2_ls_4,build_ls_0,build_ls_1,build_ls_2,build_ls_3,build_ls_4,geom,bld_eq_5,bld_eq_6,bld_eq_7,bld_eq_8,bld_eq_9,bld_eq_10,km2_eq_5,km2_eq_6,km2_eq_7,km2_eq_8,km2_eq_9,km2_eq_10,pop_eq_5,pop_eq_6,pop_eq_7,pop_eq_8,pop_eq_9,pop_eq_10,"(bld_eq, 5)","(bld_eq, 6)","(bld_eq, 7)","(bld_eq, 8)","(bld_eq, 9)","(bld_eq, 10)","(km2_eq, 5)","(km2_eq, 6)","(km2_eq, 7)","(km2_eq, 8)","(km2_eq, 9)","(km2_eq, 10)","(pop_eq, 5)","(pop_eq, 6)","(pop_eq, 7)","(pop_eq, 8)","(pop_eq, 9)","(pop_eq, 10)"
0,1.0,10.487964,2.085632,Ghazni,غزنی,AF11,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,South Eastern,جنوب ختیځه حوزه,SE,1092617,21552.89,3368.0,236035,50.69,244260,3269.02,69275,147436,2019.54,35185,49851,713.22,7977,685.95,24090,1690,769956,53122,185633,73959,9944,11866.95,1464.29,5612.20,2285.28,324.17,220345,4956,8771,1925,38,"POLYGON ((68.09305 32.39244, 68.09359 32.39044...",0,236035,1416210,708105,472070,0,0.0,4570.8,8299.2,5669.6,3003.9,0.0,0,92418,284104,282088,343198,0,0.0,1193370.0,1193370.0,1790055.0,1193370.0,0.0,0.000000,5905.251582,33826.960434,7080.383051,3009.679871,0.000000,0.000000e+00,9.475394e+04,2.376239e+06,4.562082e+05,3.433368e+05,0.000000
1,2.0,10.325135,3.794569,Nimroz,نیمروز,AF34,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,Southern,جنوبی حوزه,SR,209284,40301.16,628.0,79747,5.19,110842,12288.94,50598,75033,5096.69,37112,36683,1791.69,17245,0.00,0,0,208447,5,293,489,48,39917.80,1.08,55.06,312.99,14.23,79695,0,18,34,0,"POLYGON ((62.55349 29.38911, 62.53482 29.38706...",79315,79315,79315,0,0,0,35135.7,4304.0,448.1,0.0,0.0,0.0,185434,5156,1060,0,0,0,3566394.0,2377596.0,3566394.0,3566394.0,2377596.0,0.0,90085.093180,57369.557640,20887.948780,10638.387157,5487.702756,0.000000,1.658612e+06,1.757750e+06,2.668278e+05,2.325114e+05,1.020158e+05,0.000000
2,3.0,6.917788,0.950518,Nuristan,نورستان,AF16,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,Eastern,ختیځه حوزه,ER,260246,9579.29,323.0,6208,27.17,5876,208.04,811,4335,148.98,525,1816,57.08,293,1559.84,39018,1238,10327,34483,116469,54017,44947,-388.79,1015.85,4420.36,2450.23,2081.64,1209,1440,2677,577,305,"POLYGON ((71.19753 35.31710, 71.19756 35.31697...",0,0,0,6208,6208,0,0.0,0.0,0.0,9527.5,3.8,0.0,0,0,0,231824,15,0,0.0,0.0,488428.0,488428.0,488428.0,0.0,0.000000,0.000000,787.866446,24079.408585,3.805960,0.000000,0.000000e+00,0.000000e+00,9.381763e+04,3.375366e+06,1.546537e+01,0.000000
3,4.0,8.156241,1.265532,Uruzgan,ارزگان,AF25,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,Southern,جنوبی حوزه,SR,569711,13131.46,823.0,124085,43.39,60335,636.02,20231,43569,442.46,10531,24952,206.63,3350,1644.80,43988,2455,312575,29787,162095,54333,10920,3855.97,1088.62,5743.95,1997.25,445.67,114449,1520,6580,1527,9,"POLYGON ((65.53684 32.44911, 65.53608 32.44894...",372255,124085,248170,0,0,0,4267.3,8826.1,31.3,0.0,0.0,0.0,183809,340422,579,0,0,0,3566394.0,2377596.0,3566394.0,3566394.0,2377596.0,0.0,90085.093180,57369.557640,20887.948780,10638.387157,5487.702756,0.000000,1.658612e+06,1.757750e+06,2.668278e+05,2.325114e+05,1.020158e+05,0.000000
4,5.0,2.393424,0.185458,Kapisa,کاپیسا,AF02,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,Capital,مرکزی حوزه,CR,615265,1879.21,631.0,67797,327.41,70688,112.19,8258,35988,63.59,2460,15380,27.09,603,369.02,59995,3280,381383,17077,124025,53222,39556,482.27,95.33,725.99,294.70,280.92,59062,428,5161,3083,63,"POLYGON ((69.81884 34.80029, 69.81822 34.79999...",0,0,0,67797,67797,0,0.0,0.0,0.0,1715.8,163.2,0.0,0,0,0,406963,

In [23]:
eq_stats.rename(columns=lambda x: x[0] + '_' + str(x[1]) if isinstance(x, tuple) and isinstance(x[1], int) else x, inplace=True)
#eq_stats.rename(columns=lambda x: x[0] + '_eq_' + str(x[1]) if isinstance(x, tuple) and isinstance(x[1], int) else x, inplace=True)

In [24]:
# Convert pop figures to int
eq_stats.loc[:,'pop_eq_5':'pop_eq_10'] = eq_stats.loc[:,'pop_eq_5':'pop_eq_10'].astype(int)

# Convert building figures to int
eq_stats.loc[:,'bld_eq_5':'bld_eq_10'] = eq_stats.loc[:,'bld_eq_5':'bld_eq_10'].astype(int)

# Round area figures to 2 decimal places
eq_stats.loc[:,'km2_eq_5':'km2_eq_10'] = eq_stats.loc[:,'km2_eq_5':'km2_eq_10'].round(1)

In [25]:
# Final output
eq_stats

,tid,id,reg_en,reg_da,reg_pcode,reg_ref,regalt1en,regalt2en,regalt1da,regalt2da,adm0_en,adm0_da,adm0_pcode,date,validon,validto,shape_leng,shape_area,pop,km2,sett,build,pop_density,pop_f029,km2_f029,build_f029,pop_f121,km2_f121,build_f121,pop_f271,km2_f271,build_f271,km2_av,pop_av,build_av,pop_ls_0,pop_ls_1,pop_ls_2,pop_ls_3,pop_ls_4,km2_ls_0,km2_ls_1,km2_ls_2,km2_ls_3,km2_ls_4,build_ls_0,build_ls_1,build_ls_2,build_ls_3,build_ls_4,geom,bld_eq_5,bld_eq_6,bld_eq_7,bld_eq_8,bld_eq_9,bld_eq_10,km2_eq_5,km2_eq_6,km2_eq_7,km2_eq_8,km2_eq_9,km2_eq_10,pop_eq_5,pop_eq_6,pop_eq_7,pop_eq_8,pop_eq_9,pop_eq_10
0,1,1,Northern,سهیلی حوزه,NR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.678772,7.751351,5235170,77306.89,4714,1112341,67.72,641151,6401.16,226165,352382,3833.59,109747,123184,1318.61,33472,2982.31,95657,18817,3741116,1688,748423,687021,56920,42039.43,79.15,20452.62,13214.08,1521.61,1004581,92,57712,49139,817,"POLYGON ((68.43099 36.07082, 68.42871 36.06806...",1111586,1111586,1111586,1111586,0,0,7939.1,22789.2,37474.7,8963.6,0.0,0.0,443257,1057700,2784703,339981,0,0
1,2,2,Central Highland,لوړه مرکزی حوزه,CH,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.035766,3.100092,1224738,31641.50,4001,141838,38.71,48397,857.81,12470,35019,602.58,7939,16340,246.78,3357,3382.76,116303,17630,249666,58983,539781,271531,104775,3591.54,1473.89,16428.51,7309.05,2838.51,66744,8211,53643,12107,1133,"POLYGON ((65.61898 33.31015, 65.61822 33.30971...",141838,567352,141838,141838,0,0,5218.5,15860.5,9870.3,685.5,0.0,0.0,175887,599533,262520,72575,0,0
2,3,3,South Eastern,جنوب ختیځه حوزه,SE,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.949378,4.825809,3572212,50019.50,6882,596874,71.42,627562,6432.82,148378,377457,3987.60,73486,129614,1328.43,15912,1213.01,78842,3522,2371694,120196,585189,385661,109470,29363.46,2042.84,10686.53,6515.44,1411.23,557210,8063,16681,14778,142,"POLYGON ((68.17123 31.83539, 68.17001 31.83832...",0,1193370,1193370,1790055,1193370,0,0.0,5905.3,33827.0,7080.4,3009.7,0.0,0,94753,2376239,456208,343336,0
3,4,4,Southern,جنوبی حوزه,SR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,27.822820,17.576351,4322541,185698.95,7815,1190168,23.28,1122231,32422.42,362885,681793,16703.33,197798,268623,5295.10,60643,3774.49,85639,4739,3653686,94724,359413,185635,29081,154314.40,4401.68,15232.72,10359.61,1390.54,1162827,4962,18148,4169,62,"POLYGON ((67.58714 31.50558, 67.58714 31.50522...",3566394,2377596,3566394,3566394,2377596,0,90085.1,57369.6,20887.9,10638.4,5487.7,0.0,1658611,1757749,266827,232511,102015,0
4,5,5,Eastern,ختیځه حوزه,ER,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,10.390809,2.472625,3847087,25074.13,3555,488660,153.43,570548,1354.15,85563,333644,887.75,39404,172397,390.39,15698,2542.40,126115,3832,2359854,256418,707653,350390,172770,4126.37,3199.39,9745.48,4610.47,3392.42,437649,9825,25989,14240,957,"POLYGON ((70.00272 34.04593, 70.00179 34.04587...",0,0,488428,488428,488428,0,0.0,0.0,787.9,24079.4,3.8,0.0,0,0,93817,3375366,15,0
5,6,6,Capital,مرکزی حوزه,CR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,14.491308,3.054274,9377610,31086.15,6007,1023867,301.67,2286357,1739.51,245411,1570888,1048.60,146535,795946,415.95,66473,2986.34,305379,26367,7325615,118773,1164957,534098,234165,8471.06,2117.39,10432.25,6888.02,3177.43,896984,5777,90877,28529,1700,"POLYGON ((67.27185 33.97512, 67.26954 33.97330...",0,1023867,6143202,5119335,1023867,3071601,0.0,1527.6,7950.4,15322.3,6269.7,11.4,0,72694,449348,5791106,2150402,5102
6,7,7,North Eastern,سهیل ختیځه حوزه,NE,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,27.870669,8.141572,5463033,80700.71,6053,995154,67.69,680430,4765.26,169647,446400,3109.97,86610,207903,1227.98,28374,10800.06,402390,33810,3038725,142478,1111029,845455,325343,14775.73,5700.14,28985.39,20442.93,10796.52,

In [26]:
# Save output to database
eq_stats.to_postgis('afg_admbnda_eq_hzda_stats_v08', con, if_exists='replace')

### 5.1 QA

In [47]:
adm1 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_eq_hzda_stats_v06', con)
adm2 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_eq_hzda_stats_v07', con)
region = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_eq_hzda_stats_v08', con)

In [49]:
adm2.loc[:,'bld_eq_5':'bld_eq_10']

,bld_eq_5,bld_eq_6,bld_eq_7,bld_eq_8,bld_eq_9,bld_eq_10
0,0,0,0,3203,3203,3203
1,0,0,16797,16797,0,0
2,0,0,31267,31267,31267,0
3,0,0,8246,8246,24738,0
4,21586,0,0,0,0,0
...,...,...,...,...,...,...
396,0,0,4279,0,0,0
397,16898,33796,0,0,0,0
398,0,0,0,25638,12819,0
399,20800,20800,0,0,0,0


In [32]:
adm1

,id,shape_leng,shape_area,adm1_en,adm1_da,adm1_pcode,adm1_ref,adm1alt1en,adm1alt2en,adm1alt1da,adm1alt2da,adm0_en,adm0_da,adm0_pcode,date,validon,validto,reg_en,reg_da,reg_pcode,pop,km2,sett,build,pop_density,pop_f029,km2_f029,build_f029,pop_f121,km2_f121,build_f121,pop_f271,km2_f271,build_f271,km2_av,pop_av,build_av,pop_ls_0,pop_ls_1,pop_ls_2,pop_ls_3,pop_ls_4,km2_ls_0,km2_ls_1,km2_ls_2,km2_ls_3,km2_ls_4,build_ls_0,build_ls_1,build_ls_2,build_ls_3,build_ls_4,geom,bld_eq_5,bld_eq_6,bld_eq_7,bld_eq_8,bld_eq_9,bld_eq_10,km2_eq_5,km2_eq_6,km2_eq_7,km2_eq_8,km2_eq_9,km2_eq_10,pop_eq_5,pop_eq_6,pop_eq_7,pop_eq_8,pop_eq_9,pop_eq_10
0,1.0,10.487964,2.085632,Ghazni,غزنی,AF11,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,South Eastern,جنوب ختیځه حوزه,SE,1092617,21552.89,3368.0,236035,50.69,244260,3269.02,69275,147436,2019.54,35185,49851,713.22,7977,685.95,24090,1690,769956,53122,185633,73959,9944,11866.95,1464.29,5612.20,2285.28,324.17,220345,4956,8771,1925,38,"POLYGON ((68.09305 32.39244, 68.09359 32.39044...",0,236035,1416210,708105,472070,0,0.0,4570.8,8299.2,5669.6,3003.9,0.0,0,92418,284104,282088,343198,0
1,2.0,10.325135,3.794569,Nimroz,نیمروز,AF34,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,Southern,جنوبی حوزه,SR,209284,40301.16,628.0,79747,5.19,110842,12288.94,50598,75033,5096.69,37112,36683,1791.69,17245,0.00,0,0,208447,5,293,489,48,39917.80,1.08,55.06,312.99,14.23,79695,0,18,34,0,"POLYGON ((62.55349 29.38911, 62.53482 29.38706...",79315,79315,79315,0,0,0,35135.7,4304.0,448.1,0.0,0.0,0.0,185434,5156,1060,0,0,0
2,3.0,6.917788,0.950518,Nuristan,نورستان,AF16,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,Eastern,ختیځه حوزه,ER,260246,9579.29,323.0,6208,27.17,5876,208.04,811,4335,148.98,525,1816,57.08,293,1559.84,39018,1238,10327,34483,116469,54017,44947,-388.79,1015.85,4420.36,2450.23,2081.64,1209,1440,2677,577,305,"POLYGON ((71.19753 35.31710, 71.19756 35.31697...",0,0,0,6208,6208,0,0.0,0.0,0.0,9527.5,3.8,0.0,0,0,0,231824,15,0
3,4.0,8.156241,1.265532,Uruzgan,ارزگان,AF25,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,Southern,جنوبی حوزه,SR,569711,13131.46,823.0,124085,43.39,60335,636.02,20231,43569,442.46,10531,24952,206.63,3350,1644.80,43988,2455,312575,29787,162095,54333,10920,3855.97,1088.62,5743.95,1997.25,445.67,114449,1520,6580,1527,9,"POLYGON ((65.53684 32.44911, 65.53608 32.44894...",372255,124085,248170,0,0,0,4267.3,8826.1,31.3,0.0,0.0,0.0,183809,340422,579,0,0,0
4,5.0,2.393424,0.185458,Kapisa,کاپیسا,AF02,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,Capital,مرکزی حوزه,CR,615265,1879.21,631.0,67797,327.41,70688,112.19,8258,35988,63.59,2460,15380,27.09,603,369.02,59995,3280,381383,17077,124025,53222,39556,482.27,95.33,725.99,294.70,280.92,59062,428,5161,3083,63,"POLYGON ((69.81884 34.80029, 69.81822 34.79999...",0,0,0,67797,67797,0,0.0,0.0,0.0,1715.8,163.2,0.0,0,0,0,406963,146305,0
5,6.0,8.002717,1.810899,Paktika,پکتیکا,AF12,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,South Eastern,جنوب ختیځه حوزه,SE,600705,18887.29,1573.0,111126,31.80,106325,2268.02,29568,69907,1459.25,16055,25984,475.26,3556,114.35,3269,175,457428,3456,70724,59840,9255,13620.48,103.83,2593.44,2215.19,354.35,105855,224,2287,2734,26,"POLYGON ((68.17123 31.83539, 68.17001 31.83832...",0,111025,111025,444100,111025,0,0.0,1334.5,17023.4,362.0,5.8,0.0,0,2335,542492,7605,137,0
6,7.0,4.801921,0.794449,Kunduz,کندز,AF19,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,North Eastern,سهیل ختیځه حوزه,NE,1147660,7860.63,913.0,289473,146.00,266306,1350.82,74305,179646,907.36,40089,86759,434.42,13935,9.56,424,17,1053137,0,44868,45735,3918,6275.56,0.00,845.81,675.63,63.63,284923,0,1956,2558,36,"POLYGON ((68.99315 36.47280, 68.99071 36.47051...",0,0,0,289307,0,0,0.0,0.0,0.0,7824.3,0.0,0.0,0,0,0,1005219,0,0
7,8.0,15.864174,5.749524,Hilmand,هلمند,AF30,None,None,No

In [44]:
adm1.loc[:,'bld_eq_5':'bld_eq_10']

,bld_eq_5,bld_eq_6,bld_eq_7,bld_eq_8,bld_eq_9,bld_eq_10
0,0,236035,1416210,708105,472070,0
1,79315,79315,79315,0,0,0
2,0,0,0,6208,6208,0
3,372255,124085,248170,0,0,0
4,0,0,0,67797,67797,0
5,0,111025,111025,444100,111025,0
6,0,0,0,289307,0,0
7,505134,505134,0,0,0,0
8,88234,264702,0,0,0,0
9,0,180822,180822,180822,90411,0


In [31]:
adm = adm1
print('build_count total', adm.build.sum())
print('build_count eq', adm.loc[:, ['bld_eq_5', 'bld_eq_6', 'bld_eq_7', 'bld_eq_8', 'bld_eq_9', 'bld_eq_10']].sum().sum())
print('pop_sum total', adm.pop.sum())
print('pop_sum eq', adm.loc[:,['pop_eq_5', 'pop_eq_6', 'pop_eq_7', 'pop_eq_8', 'pop_eq_9', 'pop_eq_10']].sum().sum())
print('area total', adm.km2.sum())
print('area eq', adm.loc[:,['km2_eq_5', 'km2_eq_6', 'km2_eq_7', 'km2_eq_8', 'km2_eq_9', 'km2_eq_10']].sum().sum())

build_count total 6587646
build_count eq 32117606


AttributeError: 'function' object has no attribute 'sum'

In [ ]:
adm = adm2
print('build_count', adm.build_ls_4.sum())
print('build_count all', adm.loc[:, ['bld_eq_5', 'bld_eq_6', 'bld_eq_7', 'bld_eq_8', 'bld_eq_9', 'bld_eq_10']].sum().sum())
print('pop_sum', adm.pop_sum.sum())
print('pop_sum - excluding 0', adm.loc[:,['pop_eq_5', 'pop_eq_6', 'pop_eq_7', 'pop_eq_8', 'pop_eq_9', 'pop_eq_10']].sum().sum())
print('area_km2', adm.km2_ls_4.sum())

In [ ]:
adm = region
print('build_count', adm.build_ls_4.sum())
print('build_count all', adm.loc[:, ['bld_eq_5', 'bld_eq_6', 'bld_eq_7', 'bld_eq_8', 'bld_eq_9', 'bld_eq_10']].sum().sum())
print('pop_sum', adm.pop_sum.sum())
print('pop_sum - excluding 0', adm.loc[:,['pop_eq_5', 'pop_eq_6', 'pop_eq_7', 'pop_eq_8', 'pop_eq_9', 'pop_eq_10']].sum().sum())
print('area_km2', adm.km2_ls_4.sum())